<a href="https://colab.research.google.com/github/arjunahiran/solent-module-COM727-chatbot/blob/main/Copy_of_LungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Import Necessary Libraries***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import PIL
from google.colab import files
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.utils import image_dataset_from_directory, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, AvgPool2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
image = PIL.Image.open("/content/gdrive/MyDrive/COM726/test/adenocarcinoma/000177.png")


In [ ]:
#gpu_info = !nvidia-smi
#gpu_info = '\n'.join(gpu_info)
#if gpu_info.find('failed') >= 0:
  #print('Not connected to a GPU')
#else:
  #print(gpu_info)

In [ ]:
image

In [ ]:
"""
learning_rate = 0.0001
input_size = (224,224)
batch_size=32
monitor="val_loss"
list_model = ["ResNet101", "ResNet152", "VGG16", "EfficientNetB6", "EfficientNetB7"]
list_model_acc = []
list_model_loss = []
"""

In [ ]:
def plot_loss_acc(history) :
    """
    Plots line plots of model accuracy and model loss.

    Args:
        history (object): The history of the model.
    """
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model Accuracy')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Accuracy')
    ax[0].legend(['train', 'val'], loc='upper left')

    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model Loss')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Loss')
    ax[1].legend(['train', 'val'], loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
def eval_test(model,checkPoint_path):
    """
    Prints classification report and plots confusion matrix heatmap on the test data
    Stores accuracy and loss of models

    Args:
        model (object) : The model to be evaluated.
        checkPoint_path (str) : The path of the checkpoint
    """
    model.load_weights(checkPoint_path)
    test_evaluate = model.evaluate(test_data)
    list_model_acc.append(test_evaluate[1])
    list_model_loss.append(test_evaluate[0])
    y_pred = model.predict(test_data)
    y_pred = np.argmax(y_pred, axis=1)
    print(classification_report(y_pred,test_data.classes))
    cm = confusion_matrix(y_pred,test_data.classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    ax = disp.plot(cmap='Blues').ax_
    ax.set_title("Confusion matrix in test data")
    plt.show()



**LOAD DATA FROM DIRECTORY**

**EDA/PreProcessing on Lung cancer images**

>



In [ ]:
#To check and remove when junk folder ipynb is created
import shutil
import os

folder_path = '/content/gdrive/MyDrive/COM726/train/.ipynb_checkpoints'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print("Folder deleted successfully.")
else:
    print("Folder not found.")

In [ ]:
ROOT_DIR="/content/gdrive/MyDrive/COM726"
number_of_images = {}


# Iterate over the train, valid, and test directories
for subset in ["train", "valid", "test"]:
    subset_dir = os.path.join(ROOT_DIR, subset)
    number_of_images[subset] = {}

    # Iterate over each class directory within the subset directory
    for class_dir in os.listdir(subset_dir):
        class_path = os.path.join(subset_dir, class_dir)
        if os.path.isdir(class_path):  # Check if it's a directory
            number_of_images[subset][class_dir] = len(os.listdir(class_path))

# Print the counts
for subset, classes in number_of_images.items():
    print(f"{subset.capitalize()} data:")
    for class_name, count in classes.items():
        print(f"  {class_name}: {count} images")

**Pre processing**

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Verify the mount
!ls /content/drive/MyDrive/COM726/train

# Step 3: Define the preprocessing function
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def preprocessingImages1(path):
    """
    Input: path
    Output: preprocessed images
    """
    image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, rescale=1/255, horizontal_flip=True, fill_mode='nearest')
    #image_data = ImageDataGenerator(vertical_flip=True,zoom_range=0.1,width_shift_range=0.1,
    #height_shift_range=0.1,
    #rotation_range=10,
    #horizontal_flip=True,
    #fill_mode='nearest', rescale=1/255)


    image = image_data.flow_from_directory(directory=path, target_size=(224, 224), batch_size=32, class_mode='categorical')
    return image


In [ ]:
path = "/content/drive/MyDrive/COM726/train"
train_data = preprocessingImages1(path)

In [ ]:
def preprocessingImages2(path):
    """
    Input: path
    Output: preprocessed images
    """
    image_data = ImageDataGenerator(rescale=1/255)
    image = image_data.flow_from_directory(directory=path, target_size=(224, 224), shuffle=True, batch_size=32, class_mode='categorical')
    return image



In [ ]:
path = "/content/drive/MyDrive/COM726/test"
test_data = preprocessingImages2(path)

In [16]:
path = "/content/drive/MyDrive/COM726/valid"
valid_data = preprocessingImages2(path)

Found 72 images belonging to 4 classes.


In [ ]:
class_names = list(test_data.class_indices.keys())
plt.figure(figsize=(10, 10))

for i, (images, labels) in enumerate(train_data):
    if i == 15:
        break
    plt.subplots_adjust(hspace=0.5)
    plt.subplot(3, 5, i + 1)
    # Convert the image to uint8 and scale if necessary
    img = images[0]
    if img.max() <= 1.0:  # Check if the image is normalized between 0 and 1
        img = (img * 255).astype("uint8")
    else:
        img = img.astype("uint8")
    plt.imshow(img)
    class_index = np.argmax(labels[0])  # Use np.argmax to find the index of the class
    class_name = class_names[class_index]
    plt.title(class_name)
    plt.axis("off")
plt.tight_layout()
plt.show()

**MODEL BUILD**

In [19]:
from keras.layers import Conv2D, AvgPool2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAvgPool2D, GlobalAveragePooling2D
from keras.models import Sequential
from tensorflow.keras.applications import VGG16, EfficientNetB5
from keras.preprocessing.image import load_img, img_to_array
import keras
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import vgg16
from tensorflow.keras.models import Model

**#1 VGG16**

In [33]:
vgg16=VGG16(weights='imagenet', input_shape=(224,224,3), classes=4, include_top=False)

for layer in vgg16.layers:
    layer.trainable = False

#for layer in vgg16.layers[-4:]:  # Unfreeze the last 4 layers, for example
    #layer.trainable = True

x= Flatten()(vgg16.output)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x= Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)
model=Model(inputs=vgg16.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [42]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [43]:
#Early stopping and model check point
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience=15, verbose=1, mode="auto")

mc = ModelCheckpoint(monitor='val_accuracy', filepath='/content/drive/MyDrive/COM726/bestmodel.keras', save_best_only=True)

#es = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#mc = ModelCheckpoint('/content/drive/MyDrive/COM726/bestmodel.keras', monitor='accuracy',
cd= [es,mc]

In [44]:
#model=vgg16.model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,138,500 (80.64 MB)

 Trainable params: 6,423,812 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [46]:
hs = model.fit(train_data, steps_per_epoch=len(train_data), epochs=30, validation_data=valid_data, validation_steps=3)
#history = model.fit(train_data, steps_per_epoch=len(train_data), validation_data=valid_data, epochs=10, validation_steps=5,   callbacks=cd)
#history = model.fit(train_data, steps_per_epoch=len(train_data), validation_data=valid_data, epochs=10, validation_steps=5, callbacks=cd)

Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 458ms/step - accuracy: 0.8642 - loss: 0.5656 - val_accuracy: 0.8333 - val_loss: 0.7429
Epoch 2/30


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# Checking Accuracy using VGG16 Model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))

# Plotting Training and Validation Accuracy
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('Training & Validation Accuracy for VGG16', fontsize=16, weight='bold')

# Plotting Training and Validation Loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('Training & Validation Loss for VGG16', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')

plt.show()


In [ ]:
# Saved the model after setting these parameters vertical_flip=True, width_shift_range=0.2, height_shift_range=0.2(VGG16_lung_cancer)
model.save("/content/drive/MyDrive/COM726/vgg16_withFreezelayers.h5")
print("Saved model to disk")

# Evaluate on the test set
test_loss, test_acc = model.evaluate(valid_data)
print(f"Test Accuracy: {test_acc}")

In [ ]:
#Early stopping and model check point
es = EarlyStopping(monitor="accuracy", min_delta=0.01,
    patience=15, verbose=1, mode="auto")

mc = ModelCheckpoint(monitor="accuracy", filepath="./bestmodel.h5", verbose=1, save_best_only=True, mode='auto')

cd= [es,mc]

MODEL TRAINING
#VGG16 Model Training

In [ ]:
#Model Training
"""
hs = model.fit_generator(generator=train_data,
                         steps_per_epoch =8,
                         epochs=30,
                         verbose=1, validation_data=valid_data,
                         validation_steps =16,
                         callbacks=cd)
                         """

##hs = model.fit(train_data, epochs=30, verbose=1, validation_data=valid_data, validation_steps=16, callbacks=cd)


Achieved a Training Accuracy of 95% and Validation Acc of 81% with VGG16, now lets move to InceptionResNetV2 model

In [ ]:
classes=list(train_data.class_indices.keys())
class_count=len(classes)

print(class_count)

In [ ]:
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import *


**#2 InceptionResNetV2**

In [ ]:
model_name='InceptionResNetV2'
base_model=tf.keras.applications.InceptionResNetV2(include_top=False, weights="imagenet",input_shape=(224, 224, 3), pooling='max')
x=base_model.output

In [ ]:
#Early stopping and model check point for InceptionResnetV2 model
es = EarlyStopping(monitor="val_loss", patience=10, verbose=1)

mc = ModelCheckpoint(monitor="val_loss", filepath="./bestmodel.h5", verbose=1, save_best_only=True, mode='auto')

cd= [es,mc]

In [ ]:
# Add new layers
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)

x = Dense(256, kernel_regularizer=regularizers.l2(0.02), activity_regularizer=regularizers.l1(0.01),
          bias_regularizer=regularizers.l1(0.01), activation='relu')(x)
#x = Dense(256, kernel_regularizer = regularizers.l2(0.016),activity_regularizer=regularizers.l1(0.006),
                #bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x = Dropout(rate=0.45, seed=123)(x)
output = Dense(4, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  #Precision(name='precision'),**/
                     # Recall(name='recall'),**/
                      #AUC(name='auc'),**/
                     #RootMeanSquaredError(name='root_mean_squared_error')])**/
#model.compile(Adamax(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#early_stopping=EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=30,
    callbacks=cd)
#model.save_weights("model.h5")
model.save("InceptionResNetV2.hd5")
print("Saved model to disk")

**InceptionResnet Model achived 83% of training accuracy however high loss %**

In [ ]:
#Checking Accuracy using InceptionResnet Model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)

plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')


plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('Training & Validation Acc.', fontsize=16, weight='bold')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')
plt.show()

**#3 EfficientNetB5 Model**

In [ ]:
#Model Creation of EfficientNetB5
# Using the EfficientNetB5 pre-trained model as a base model (without the fully connected layers)
base_model = tf.keras.applications.efficientnet.EfficientNetB5(
    include_top=False,     # Exclude the fully connected layers
    weights="imagenet",    # Load pre-trained ImageNet weights
    input_shape=(224, 224, 3),  # Specify the input shape for the model
    pooling='max'           # Use global max pooling as the final pooling layer
)

# Constructing the complete model using Sequential API
model = Sequential([
    base_model,  # EfficientNetB5 as the base model
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),  # Batch normalization layer
    Dense(256,
          kernel_regularizer=regularizers.l2(l=0.016),
          activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006),
          activation='relu'),  # Dense layer with regularization and ReLU activation
    Dropout(rate=0.45, seed=123),  # Dropout layer for regularization
    Dense(4, activation='softmax')  # Output layer with softmax activation for multi-class classification
])

# Compile the model with specified optimizer, loss function, and evaluation metric
model.compile(
    optimizer=Adamax(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [ ]:
# Display a summary of the model architecture
model.summary()

In [ ]:
# Retrieve the configuration of the optimizer used in the EfficientNetB5 base model
model.optimizer.get_config()

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
mc = ModelCheckpoint('model_weights_efficient_B5_2.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min',
                             verbose=1)

cd= [es,mc]

In [ ]:
history = model.fit(x= train_data,
                    steps_per_epoch = 20,
                    epochs= 30,
                    callbacks=[cd],
                    validation_data = valid_data)

# Check for Missing or Corrupted Files

In [ ]:
#Check for Missing or Corrupted Files
import os
from PIL import Image

def check_for_corrupted_images(directory):
    corrupted_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png', 'bmp', 'tiff')):
                file_path = os.path.join(root, file)
                try:
                    img = Image.open(file_path)
                    img.verify()  # Verify that it is an image
                except (IOError, SyntaxError) as e:
                    print(f"Corrupted file found: {file_path}")
                    corrupted_files.append(file_path)
    return corrupted_files

train_data = '/content/gdrive/MyDrive/COM726/train'
valid_data = '/content/gdrive/MyDrive/COM726/valid'
test_data = '/content/gdrive/MyDrive/COM726/test'

corrupted_train = check_for_corrupted_images(train_data)
corrupted_valid = check_for_corrupted_images(valid_data)
corrupted_test = check_for_corrupted_images(test_data)


**Check Image Quality**

In [ ]:
def is_low_contrast_color(image, threshold=0.05):
    # Apply is_low_contrast to each channel separately
    for i in range(image.shape[-1]):  # Assuming image is in (height, width, channels) format
        if exposure.is_low_contrast(image[..., i], fraction_threshold=threshold):
            return True
    return False

def check_image_quality(generator, num_batches=1):
    low_quality_images = []
    for _ in range(num_batches):
        #images, _ = next(generator)
        for img in images:
            img = np.array(img).astype(np.uint8)
            if img.ndim == 3:  # Check if image has color channels
                if is_low_contrast_color(img):
                    low_quality_images.append(img)
            else:
                if exposure.is_low_contrast(img):
                    low_quality_images.append(img)
    return low_quality_images

# Assuming train_data, valid_data, and test_data are defined and are generators

# Check low quality images
low_quality_train = check_image_quality(train_data)
low_quality_valid = check_image_quality(valid_data)
low_quality_test = check_image_quality(test_data)

print(f"Low quality images in training set: {len(low_quality_train)}")
print(f"Low quality images in validation set: {len(low_quality_valid)}")
print(f"Low quality images in test set: {len(low_quality_test)}")

# Visualize low quality images if any
def visualize_low_quality_images(images, title):
    plt.figure(figsize=(15, 15))
    num_images = len(images)
    for i in range(min(num_images, 5)):
        plt.subplot(1, 5, i + 1)
        plt.imshow(images[i])
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_low_quality_images(low_quality_train, 'Low Quality - Train')
visualize_low_quality_images(low_quality_valid, 'Low Quality - Valid')
visualize_low_quality_images(low_quality_test, 'Low Quality - Test')

In [ ]:
# Displaying the model performance
def model_performance(history, Epochs):
    # Define needed variables
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']

    Epochs = [i+1 for i in range(len(tr_acc))]

    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')

    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.show()


# Evaluate the model
def model_evaluation(model):
    train_score = model.evaluate(train_gen, verbose= 1)
    valid_score = model.evaluate(valid_gen, verbose= 1)
    test_score = model.evaluate(test_gen, verbose= 1)

    print("Train Loss: ", train_score[0])
    print("Train Accuracy: ", train_score[1])
    print('-' * 20)
    print("Validation Loss: ", valid_score[0])
    print("Validation Accuracy: ", valid_score[1])
    print('-' * 20)
    print("Test Loss: ", test_score[0])
    print("Test Accuracy: ", test_score[1])


# Get Predictions
def get_pred(model, test_gen):

    preds = model.predict(test_gen)
    y_pred = np.argmax(preds, axis = 1)

    return y_pred


# Confusion Matrix
def plot_confusion_matrix(test_gen, y_pred):

    g_dict = test_gen.class_indices
    classes = list(g_dict.keys())

    # Display the confusion matrix
    cm = confusion_matrix(test_gen.classes, y_pred)

    plt.figure(figsize= (10, 10))
    plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation= 45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')


    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

    plt.show()


# Defining a convolutional NN block for a sequential CNN model
def conv_block(filters, act='relu'):

    block = Sequential()
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D())

    return block


# Defining a dense NN block for a sequential CNN model
def dense_block(units, dropout_rate, act='relu'):

    block = Sequential()
    block.add(Dense(units, activation=act))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))

    return block